# NuGetConsume

NGC is an API to depend on NuGet packages in Dyalog APL.

It takes care of installing from NuGet and some of the managing of `⎕USING` statements.

`⎕using←'NSName,[thing]'`
thing is either name of Assembly or path to file

the bridge can call Assembly.Load and "lots of the time?" do the gymnastics to bring in the assembly

## Features
- add packages
- get path to .dll for `⎕USING` (additional namespaces might not be possible automatically)
- maybe how to export .csproj?
    - or just list of .NET/NuGet dependencies as part of ]Cider/Tatin config?
- does NGC need stuff for when we export APL as .NET stuff?
- `NGC.Export'publish'` will export DLLs
    - `NGC.Export'publish' 'platform'`
- `NGC.GetPath` (`FullPath` in nugetconsum)
- retarget existing project? TBD - might only be able to take packages list and attempt create new .NET project

## How to ship APL as a (NuGet/.NET) library
- provide bridge and other DLLs/exes etc. as dependencies
- `⎕NQ'Bind'` for export assembly needs dependency info

#todo shipping APL generally:
- end user non-APLer app
- C# dev
- APL dev e.g. UCMD

## Todo
- check can `⎕NEW` on existing .csproj
- document info on platform compatibility
- can we get DLL name from package info? esp. if different from package name...
- can we get assembly namespaces from package info (I think Bjørn said not necessarily)
- tmp path for .csproj by default?
- can we have it *not* be a class?
    - `⎕NEW thing` to carry around info seems nice
    - but end up not hiding anything since all my methods are public?
    - but what other way to hold the info? pass a namespace around to functions? 
- do we need `Hook/NuGetPackageHandler` methods if we do dependencies from `project.assets.json`?
- should user supply Assebly Namespaces at `AddPackage` or when setting `⎕USING`?
- `⎕NULL in [3] will add & install the package, but not add it to ⎕USING` ?
- can we get chromedriver exe location from package info?

## .NET projects
NGC uses the `dotnet` Command Line Interface (CLI) to set up a .NET project which manages obtaining compatible versions of NuGet packages and their dependencies.

The project is defined in a **.csproj** file, while dependencies and paths to assembly DLLs are listed in **obj/project.assets.json**.

## .NET Versions
Dyalog can be configured to use various .NET versions [*Dyalog Forums post*](https://forums.dyalog.com/viewtopic.php?f=22&t=1863)

> kind of bleh  
> Developers need to install the **SDK** in order to target dotnet .csproj projects  
> alternative?  
> learn the nuget API and manage deps ourselves... also bleh

**Officially supported .NET versions:**
|Dyalog version|.NET version|.NET Framework (Windows only)
|---|---|---|
|17.1|3.1|4.8|
|18.2|3.1|4.8|
|19.0|6.0|4.8|

NGC will create and export .NET packages under a single .NET project targeting the currently running .NET version.

NuGet Gallery package pages ([example: Clock](https://www.nuget.org/packages/Clock#supportedframeworks-body-tab)) list compatible versions under "frameworks" tab. **netstandard** is supported across .NET versions, while **netN** for some number **N** corresponds to a particular .NET version.

> **Note:** `dotnet` CLI seems just NETCore/New .NET or NetStandard - need to force .NET Framework?  
> `dotnet new classlib --target-framework-override net48 -o /tmp/myapp`

## Usage
Create an instance of the NGC class.

In [48]:
]create NGC "C:\g\NuGetConsume\APLSource"

ERRORS ENCOUNTERED: ⎕SE.Link.Create: Cannot link a non-empty namespace to a non-empty directory: #.NGC ←→ C:\g\NuGetConsume\APLSource

In [117]:
]link.refresh #.NGC

Imported: #.NGC ← C:\g\NuGetConsume\APLSource

**Note:** Jupyter kernel doesn't seem to play nice with FSW - no idea why

In [118]:
ngc←⎕NEW NGC.NGC'/tmp/myapp3'

In [119]:
ngc.AddPackages 'Selenium.WebDriver' '4.1.0'

***Note:*** Jupyter kernel can't print asynchronous output... [issue 82](https://github.com/Dyalog/dyalog-jupyter-kernel/issues/82)

Or maybe unrelated... why does AddPackages output work?

In [113]:
⎕←ngc.project_dir
⎕←ngc.dotnet_version
 'dotnet new classlib -f ',ngc.dotnet_version,' -o ',ngc.project_dir

/tmp/myapp

net48

dotnet new classlib -f net48 -o /tmp/myapp

In [114]:
ngc.AddPackages 'Clock'

Determining projects to restore...
 Writing C:\Users\rpark\AppData\Local\Temp\tmpCAF7.tmp
info : Adding PackageReference for package 'Clock' into project 'C:\tmp\myapp\myapp.csproj'.
info : GET https://api.nuget.org/v3/registration5-gz-semver2/clock/index.json
info : OK https://api.nuget.org/v3/registration5-gz-semver2/clock/index.json 133ms
info : Restoring packages for C:\tmp\myapp\myapp.csproj...
info : Package 'Clock' is compatible with all the specified frameworks in project 'C:\tmp\myapp\myapp.csproj'.
info : PackageReference for package 'Clock' version '1.0.3' added to file 'C:\tmp\myapp\myapp.csproj'.
info : Generating MSBuild file C:\tmp\myapp\obj\myapp.csproj.nuget.g.props.
info : Writing assets file to disk. Path: C:\tmp\myapp\obj\project.assets.json
log : Restored C:\tmp\myapp\myapp.csproj (in 117 ms).

In [115]:
⎕USING←''
⎕USING,←⎕←ngc.GetUsing'Clock'
Clock
Clock.UtcNow

LENGTH ERROR: Mismatched left and right argument shapes
Using[6] tgt←p=1+⍸'targets' 'net6.0'⍷n ⋄ pid←⍸p=tgt
              ∧
VALUE ERROR: Undefined name: Clock
      Clock
      ∧
VALUE ERROR: Undefined name: Clock
      Clock.UtcNow
      ∧


## Parque.Net

In [80]:
ngc.AddPackages 'Parquet.Net' '4.15.0'

Determining projects to restore...
 Writing C:\Users\rpark\AppData\Local\Temp\tmpC2A7.tmp
info : Adding PackageReference for package 'Parquet.Net' into project 'C:\tmp\myapp\myapp.csproj'.
info : Restoring packages for C:\tmp\myapp\myapp.csproj...
warn : NU1701: Package 'Html2Xhtml 1.1.2.4' was restored using '.NETFramework,Version=v4.6.1, .NETFramework,Version=v4.6.2, .NETFramework,Version=v4.7, .NETFramework,Version=v4.7.1, .NETFramework,Version=v4.7.2, .NETFramework,Version=v4.8' instead of the project target framework 'net6.0'. This package may not be fully compatible with your project.
info : Package 'Parquet.Net' is compatible with all the specified frameworks in project 'C:\tmp\myapp\myapp.csproj'.
info : PackageReference for package 'Parquet.Net' version '4.15.0' updated in file 'C:\tmp\myapp\myapp.csproj'.
info : Assets file has not changed. Skipping assets file writing. Path: C:\tmp\myapp\obj\project.assets.json
log : Restored C:\tmp\myapp\myapp.csproj (in 183 ms).

In [93]:
⍝⎕USING←'System.IO,System.IO.FileSystem,System.IO.Stream'
⍪ngc.GetUsing'Parquet.Net'   ⍝ uh oh

┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/netcoreapp2.1/Apache.Arrow.dll │
├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net6.0/IronCompress.dll │
├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/netstandard2.0/Microsoft.Data.Analysis.dll │
├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net5.0/Microsoft.IO.RecyclableMemoryStream.dll │
├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/netstandard2.0/Microsoft.ML.DataView.dll │
├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net6.0/Parquet.dll │
├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net6.0/Snappier.dll │
├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/netstandard2.0/System.Collections.Immutable.dll │
├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net6.0/System.Runtime.CompilerServices.Unsafe.dll│
├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net6.0/ZstdSharp.dll │
└───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [99]:
⍝⎕USING←'System.IO,System.IO.dll,System.IO.FileSystem,System.IO.Stream'
⎕USING←'' 'System' 'System.IO'
⎕USING,←ngc.GetUsing'Parquet.Net'
Parquet

Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\netcoreapp2.1\Apache.Arrow.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net6.0\IronCompress.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\netstandard2.0\Microsoft.Data.Analysis.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net5.0\Microsoft.IO.RecyclableMemoryStream.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\netstandard2.0\Microsoft.ML.DataView.dll' or one of its dependencies. The system cannot find